In [43]:
import numpy as np

# Input: sequence of images: b/w
def create_input(l=32,b=48,t=48):
    # creates input of shape (t,l,b)
    input = np.zeros((t,l,b),dtype=np.bool)
    for i in range(t):
        input[i,:,(i*l)//t]=True
    return input

def delay(x,tau):
    return np.concatenate((np.zeros(tau),x[tau:]))

def medulla(input,tau):
    # input from pair of ommatidia to HR detector pair. Shape: (2,t)
    output = np.zeros((4,t))
    
    output[1,:] = delay(input[1,:],tau['1'])*delay(input[2,:],tau['b'])
    output[2,:] = delay(input[1,:],tau['b'])*delay(input[2,:],tau['1'])
    output[3,:] = delay(input[1,:],tau['2'])*delay(input[2,:],tau['b'])
    output[4,:] = delay(input[1,:],tau['b'])*delay(input[2,:],tau['2'])
    
    return output

def lobulla(input):
    # input shape: (l*b//2,4,t)
    sum_out = np.sum(input, axis=0)
    return np.concatenate(([sum_out[0]-sum_out[1]],[sum_out[2]-sum_out[3]]),axis=0)   # shape: (2,t)

def AVDU # TODO Vibhu :)